# Quickstart: Agentic retrieval in Azure AI Search

Use this notebook to get started with [agentic retrieval](https://learn.microsoft.com/azure/search/search-agentic-retrieval-concept) in Azure AI Search, which integrates conversation history and large language models (LLMs) on Azure OpenAI to plan, retrieve, and synthesize complex queries.

Steps in this notebook include:

+ Creating an `earth_at_night` search index.

+ Loading the index with documents from a GitHub URL.

+ Creating an `earth-search-agent` in Azure AI Search that points to an LLM for query planning.

+ Using the agent to fetch and rank relevant information from the index.

+ Generating answers using the Azure OpenAI client.

This notebook provides a high-level demonstration of agentic retrieval. For more detailed guidance, see [Quickstart: Run agentic retrieval in Azure AI Search](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval).

## Prerequisites

+ An [Azure AI Search service](https://learn.microsoft.com/azure/search/search-create-service-portal) on the Basic tier or higher with [semantic ranker enabled](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable).

+ An [Azure OpenAI resource](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource).

+ A [supported model](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models) deployed to your Azure OpenAI resource. This notebook uses `gpt-4o-mini`.

## Configure access

This notebook assumes authentication and authorization using Microsoft Entra ID and role assignments. It also assumes that you run the code from your local device.

To configure role-based access:

1. Sign in to the [Azure portal](https://portal.azure.com).

1. [Enable role-based access](https://learn.microsoft.com/azure/search/search-security-enable-roles) on your Azure AI Search service.

1. [Create a system-assigned managed identity](https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources#create-a-system-managed-identity) on your Azure AI Search service.

1. On your Azure AI Search service, [assign the following roles](https://learn.microsoft.com/azure/search/search-security-rbac#how-to-assign-roles-in-the-azure-portal) to yourself.

   + **Search Service Contributor**

   + **Search Index Data Contributor**

   + **Search Index Data Reader**

1. On your Azure OpenAI resource, assign **Cognitive Services User** to the managed identity of your search service.

## Set up connections

The `sample.env` file contains environment variables for connections to Azure AI Search and Azure OpenAI. Agentic retrieval requires these connections for document retrieval, query planning, query execution, and answer generation.

To set up connections:

1. Sign in to the [Azure portal](https://portal.azure.com).

2. Retrieve the endpoints for both Azure AI Search and Azure OpenAI.

3. Save the `sample.env` file as `.env` on your local device.

4. Update the `.env` file with the retrieved endpoints.

## Install packages and load connections

This step installs the packages for this notebook and establishes connections to Azure AI Search and Azure OpenAI.

In [ ]:
#r "nuget: Azure.Search.Documents, 11.7.0-beta.7"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget:dotenv.net, 4.0.0"

In [ ]:
using dotenv.net;
using Azure.Identity;

// .env should be in the same directory as this notebook
DotEnv.Load(options: new DotEnvOptions(envFilePaths: new[] { ".env" }, ignoreExceptions: false));

// Get environment variables with defaults where appropriate
string answerModel = Environment.GetEnvironmentVariable("ANSWER_MODEL") ?? "gpt-5";
string endpoint = Environment.GetEnvironmentVariable("AZURE_SEARCH_ENDPOINT") 
    ?? throw new InvalidOperationException("AZURE_SEARCH_ENDPOINT is not set.");
var credential = new DefaultAzureCredential();

string indexName = Environment.GetEnvironmentVariable("AZURE_SEARCH_INDEX") ?? "earth_at_night";
string knowledgeSourceName = Environment.GetEnvironmentVariable("AZURE_SEARCH_KNOWLEDGE_SOURCE_NAME") ?? "earth-at-night-ks";
string azureOpenAiEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT") 
    ?? throw new InvalidOperationException("AZURE_OPENAI_ENDPOINT is not set.");
string azureOpenAiGptDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_GPT_DEPLOYMENT") ?? "gpt-5";
string azureOpenAiGptModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_GPT_MODEL") ?? "gpt-5";
string azureOpenAiApiVersion = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_VERSION") ?? "2025-03-01-preview";
string azureOpenAiEmbeddingDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDING_DEPLOYMENT") ?? "text-embedding-3-large";
string azureOpenAiEmbeddingModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDING_MODEL") ?? "text-embedding-3-large";
string agentName = Environment.GetEnvironmentVariable("AZURE_SEARCH_AGENT_NAME") ?? "earth-search-agent";
string apiVersion = "2025-05-01-Preview";

## Create an index in Azure AI Search

This step creates a search index that contains plain text and vector content. You can use an existing index, but it must meet the criteria for [agentic retrieval workloads](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index). The primary schema requirement is a semantic configuration with a `default_configuration_name`.

In [ ]:
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;

// Define the fields for the index
var fields = new List<SearchField>
{
    new SimpleField("id", SearchFieldDataType.String) { IsKey = true, IsFilterable = true, IsSortable = true, IsFacetable = true },
    new SearchField("page_chunk", SearchFieldDataType.String) { IsFilterable = false, IsSortable = false, IsFacetable = false },
    new SearchField("page_embedding_text_3_large", SearchFieldDataType.Collection(SearchFieldDataType.Single)) { VectorSearchDimensions = 3072, VectorSearchProfileName = "hnsw_text_3_large" },
    new SimpleField("page_number", SearchFieldDataType.Int32) { IsFilterable = true, IsSortable = true, IsFacetable = true }
};

// Define the vectorizer
var vectorizer = new AzureOpenAIVectorizer(vectorizerName: "azure_openai_text_3_large")
{
    Parameters = new AzureOpenAIVectorizerParameters
    {
        ResourceUri = new Uri(azureOpenAiEndpoint),
        DeploymentName = azureOpenAiEmbeddingDeployment,
        ModelName = azureOpenAiEmbeddingModel
    }
};

// Define the vector search profile and algorithm
var vectorSearch = new VectorSearch()
{
    Profiles =
    {
        new VectorSearchProfile(
            name: "hnsw_text_3_large",
            algorithmConfigurationName: "alg"
        )
        {
            VectorizerName = "azure_openai_text_3_large"
        }
    },
    Algorithms =
    {
        new HnswAlgorithmConfiguration(name: "alg")
    },
    Vectorizers =
    {
        vectorizer
    }
};

// Define semantic configuration
var semanticConfig = new SemanticConfiguration(
    name: "semantic_config",
    prioritizedFields: new SemanticPrioritizedFields
    {
        ContentFields = { new SemanticField("page_chunk") }
    }
);

var semanticSearch = new SemanticSearch()
{
    DefaultConfigurationName = "semantic_config",
    Configurations =
    {
        semanticConfig
    }
};

// Create the index
var index = new SearchIndex(indexName)
{
    Fields = fields,
    VectorSearch = vectorSearch,
    SemanticSearch = semanticSearch
};

// Create the index client and create or update the index
var indexClient = new SearchIndexClient(new Uri(endpoint), credential);
await indexClient.CreateOrUpdateIndexAsync(index);

Console.WriteLine($"Index '{indexName}' created or updated successfully");

## Upload sample documents

This notebook uses data from NASA's Earth at Night e-book. The data is retrieved from the [azure-search-sample-data](https://github.com/Azure-Samples/azure-search-sample-data) repository on GitHub and passed to the search client for indexing.

In [ ]:
using System.Net.Http;
using System.Text.Json;
using Azure.Search.Documents;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Models;

// Download the documents from the GitHub URL
string url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json";
var httpClient = new HttpClient();
var response = await httpClient.GetAsync(url);
response.EnsureSuccessStatusCode();
var json = await response.Content.ReadAsStringAsync();

var documents = JsonSerializer.Deserialize<List<Dictionary<string, object>>>(json);
var searchClient = new SearchClient(new Uri(endpoint), indexName, credential);
var searchIndexingBufferedSender = new SearchIndexingBufferedSender<Dictionary<string, object>>(
    searchClient,
    new SearchIndexingBufferedSenderOptions<Dictionary<string, object>>
    {
        KeyFieldAccessor = doc => doc["id"].ToString(),
    }
);

await searchIndexingBufferedSender.UploadDocumentsAsync(documents);
await searchIndexingBufferedSender.FlushAsync();

Console.WriteLine($"Documents uploaded to index '{indexName}'");

## Create a knowledge source

This step creates a knowledge source that targets the index you previously created. In the next step, you create a knowledge agent that uses the knowledge source to orchestrate agentic retrieval.

In [ ]:
using Azure.Search.Documents.Indexes.Models;

var indexKnowledgeSource = new SearchIndexKnowledgeSource(
    name: knowledgeSourceName,
    searchIndexParameters: new SearchIndexKnowledgeSourceParameters(searchIndexName: indexName)
    {
        SourceDataSelect = "id,page_chunk,page_number"
    }
);

await indexClient.CreateOrUpdateKnowledgeSourceAsync(indexKnowledgeSource);

## Create a knowledge agent


This step creates a knowledge agent, which acts as a wrapper for your knowledge source and LLM deployment.

ExtractiveData is the default modality and returns content from your knowledge sources without generative alteration. However, this quickstart uses the AnswerSynthesis modality for LLM-generated answers that cite the retrieved content.

In [ ]:
using Azure.Search.Documents.Indexes.Models;

var openAiParameters = new AzureOpenAIVectorizerParameters
{
    ResourceUri = new Uri(azureOpenAiEndpoint),
    DeploymentName = azureOpenAiGptDeployment,
    ModelName = azureOpenAiGptModel
};

var agentModel = new KnowledgeAgentAzureOpenAIModel(azureOpenAIParameters: openAiParameters);

var outputConfig = new KnowledgeAgentOutputConfiguration
{
    Modality = KnowledgeAgentOutputConfigurationModality.AnswerSynthesis,
    IncludeActivity = true
};

// Create the knowledge agent
var agent = new KnowledgeAgent(
    name: agentName,
    models: new[] { agentModel },
    knowledgeSources: new KnowledgeSourceReference[] { new KnowledgeSourceReference(knowledgeSourceName) { IncludeReferences = true, IncludeReferenceSourceData = true }}
)
{
    OutputConfiguration = outputConfig
};
await indexClient.CreateOrUpdateKnowledgeAgentAsync(agent);
Console.WriteLine($"Search agent '{agentName}' created or updated successfully");

## Set up messages

Messages are the input for the retrieval route and contain the conversation history. Each message includes a `role` that indicates its origin, such as `assistant` or `user`, and `content` in natural language. The LLM you use determines which roles are valid.

In [ ]:
string instructions = @"
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with ""I don't know"".
";

var messages = new List<Dictionary<string, string>>
{
    new Dictionary<string, string>
    {
        { "role", "system" },
        { "content", instructions }
    }
};

## Use agentic retrieval to fetch results

This step runs the retrieval pipeline to extract relevant information from your search index. Based on the messages and parameters on the retrieval request, the LLM:

1. Analyzes the entire conversation history to determine the underlying information need.

1. Breaks down the compound user query into focused subqueries.
 
1. Runs each subquery simultaneously against text fields and vector embeddings in your index.

1. Uses semantic ranker to rerank the results of all subqueries.

1. Merges the results into a single string.

In [ ]:
using Azure.Search.Documents.Agents;
using Azure.Search.Documents.Agents.Models;

var agentClient = new KnowledgeAgentRetrievalClient(
    endpoint: new Uri(endpoint),
    agentName: agentName,
    tokenCredential: new DefaultAzureCredential()
);

messages.Add(new Dictionary<string, string>
{
    { "role", "user" },
    { "content", @"
Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
" }
});

var retrievalResult = await agentClient.RetrieveAsync(
    retrievalRequest: new KnowledgeAgentRetrievalRequest(
            messages: messages
                .Where(message => message["role"] != "system")
                .Select(
                    message => new KnowledgeAgentMessage(content: new[] { new KnowledgeAgentMessageTextContent(message["content"]) })  { Role = message["role"] }
                )
                .ToList()
            )
    );

messages.Add(new Dictionary<string, string>
{
    { "role", "assistant" },
    { "content", (retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text }
});

### Review the retrieval response, activity, and results

Because your knowledge agent is configured for answer synthesis, the retrieval response contains the following values:

+ An LLM-generated answer to the query that cites the retrieved documents.

+ Detailed planning and execution information, including subqueries, reranking decisions, and intermediate steps.

+ Source documents and chunks that contributed to the answer.

In [ ]:
(retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text 

In [ ]:
Console.WriteLine("Activities:");
foreach (var activity in retrievalResult.Value.Activity)
{
    Console.WriteLine($"Activity Type: {activity.GetType().Name}");
    string json = JsonSerializer.Serialize(
        activity,
        activity.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Console.WriteLine("Results");
foreach (var reference in retrievalResult.Value.References)
{
    Console.WriteLine($"Reference Type: {reference.GetType().Name}");
    string json = JsonSerializer.Serialize(
        reference,
        reference.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

## Continue the conversation

This step continues the conversation with the search agent, building upon the previous messages and queries to retrieve relevant information from your search index.

In [ ]:
messages.Add(new Dictionary<string, string>
{
    { "role", "user" },
    { "content", "How do I find lava at night?" }
});


var retrievalResult = await agentClient.RetrieveAsync(
    retrievalRequest: new KnowledgeAgentRetrievalRequest(
            messages: messages
                .Where(message => message["role"] != "system")
                .Select(
                    message => new KnowledgeAgentMessage(content: new[] { new KnowledgeAgentMessageTextContent(message["content"]) })  { Role = message["role"] }
                )
                .ToList()
            )
    );

messages.Add(new Dictionary<string, string>
{
    { "role", "assistant" },
    { "content", (retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text }
});


### Review the retrieval response, activity, and results

In [ ]:
(retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text 

In [ ]:
Console.WriteLine("Activities:");
foreach (var activity in retrievalResult.Value.Activity)
{
    Console.WriteLine($"Activity Type: {activity.GetType().Name}");
    string json = JsonSerializer.Serialize(
        activity,
        activity.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Console.WriteLine("Results");
foreach (var reference in retrievalResult.Value.References)
{
    Console.WriteLine($"Reference Type: {reference.GetType().Name}");
    string json = JsonSerializer.Serialize(
        reference,
        reference.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

## Clean up objects and resources

If you no longer need Azure AI Search or Azure OpenAI, delete them from your Azure subscription. You can also start over by deleting individual objects.

### Delete the search agent

In [ ]:
await indexClient.DeleteKnowledgeAgentAsync(agentName);
System.Console.WriteLine($"Search agent '{agentName}' deleted successfully");

### Delete the search index

In [ ]:
await indexClient.DeleteIndexAsync(indexName);
System.Console.WriteLine($"Index '{indexName}' deleted successfully");